In [ ]:
!pip install requests
!pip install bs4
!pip install sklearn
!pip install pandas
!pip install matplotlib


Metacritic 사이트로 부터 데이터 크롤링
전체를 한번에 긁어오면 오류가 발생해, 1995~2022년을 년도 별로 나누어 크롤링 하였다.

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
user_agent = {'User-agent': 'Mozilla/5.0'}
links=[]
data_list=[]
first_row = ['name', 'platform', 'metascore', 'userscore', 'developer', 'genre']
def get_href():
  links = []
  for i in range(0, 10) :
    url = "https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=2008&distribution=&sort=desc&view=detailed&page="+str(i)
    html = requests.get(url, headers=user_agent)
    soup = BeautifulSoup(html.content, 'html.parser')
    td = soup.find_all('td', attrs = {'class':'clamp-summary-wrap'})
    for k in td:
      a = k.find("a", class_="title")['href']
      links.append("https://www.metacritic.com/"+a)
  return links
def get_data(links):
  check = 0
  temp_list = []
  for link in links:
    temp_genre=''
    url = link
    html = requests.get(url, headers=user_agent)
    soup = BeautifulSoup(html.content, 'html.parser')
    try:
      name = soup.find("div", class_="product_title").find("h1").get_text()
      try:
        platform = soup.find("span", class_="platform").get_text()
      except:
        platform = "tbd"
      try:
        metascore = soup.find("div", class_="metascore_wrap highlight_metascore").find("span", itemprop="ratingValue").get_text()
      except:
        metascore = "tbd"
        userscore = soup.select('a.metascore_anchor')[0].text
        check = 1
      if check != 1:
        userscore = soup.select('a.metascore_anchor')[1].text
      try:
        developer = soup.find("li", class_="summary_detail developer").find("span", class_="data").get_text()
      except:
        developer ="tbd"
      genre = soup.find("li", class_="summary_detail product_genre").find_all("span", class_="data")
      for i in range(len(genre)):
        temp_genre += genre[i].get_text()
      temp_list.append([name, platform, metascore, userscore, developer, temp_genre])
    except:
      pass
  return temp_list
def make_csv(data_list):
  with open('2008topgame.csv', 'w', encoding='utf-8', newline='') as file:
    csvfile = csv.writer(file)
    csvfile.writerow(first_row)
    for row in data_list:
      csvfile.writerow(row)
links = get_href()
temp = get_data(links)
data_list += temp
make_csv(data_list)

  









In [ ]:
from operator import index
import os
from glob import glob
import pandas as pd

file_names = glob('./crawling_data/*.csv')
total = pd.DataFrame()
for file_name in file_names:
    temp = pd.read_csv(file_name, encoding='utf-8')
    total=pd.concat([total, temp])
total.to_csv('./crawling_data/total.csv', index=False)

total.csv 파일 불러와서 중복 제거, tbd같은 데이터 0으로 변환, metascore, userscore 각각 int와 float로 변환

In [ ]:
data = pd.read_csv('./crawling_data/total.csv')
data = data.drop_duplicates(['name', 'platform', 'metascore', 'userscore'], ignore_index = True)
data['metascore'] = data['metascore'].str.replace('tbd', '0')
data['userscore'] = data['userscore'].str.replace('tbd', '0')
data['platform'] = data['platform'].str.strip()
data['metascore'] = data['metascore'].astype(int)
data['userscore'] = data['userscore'].astype(float)                                		    






장르, 개발사, 게임 플랫폼을 합쳐서 feature라는 하나의 문자열 생성
metascore와 userscore를 합친 totalscore라는 점수 생성

In [ ]:
data['feature'] = data['genre']+data['developer']+data['platform']
data['totalscore'] = data['metascore']+data['userscore']*10

TF-IDF 수행

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.decomposition import PCA
from sklearn.utils.extmath import randomized_svd
import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 5), stop_words='english')
tfidf_matrix = tf.fit_transform(data['feature'])
print(tfidf_matrix.shape)
print(tfidf_matrix.toarray())


Randomized svd로 tfidf matrix 분해

In [ ]:
U, s, V = randomized_svd(tfidf_matrix, n_components = 200)


left singular matrix (game x latent factor)를 이용해 코사인 유사도 측정

In [ ]:
cosine_sim = linear_kernel(U)
cosine_sim = cosine_sim.argsort()[:, ::-1]
print(cosine_sim[:10, :10])


cosine similarity를 바탕으로 유사도 상위 10개 게임 출력 (추후 내가 맞는 플랫폼 필터링과 메타스코어, 유저 평점 적용 예정)

In [ ]:
def find_sim_game(data, cosine_sim, game_name, game_platform, user_platform, top=10):
    game_a = data[(data['name'] == game_name)&(data['platform']== game_platform)]
    game_name_check = data[data['name'] == game_name]
    platform = data[data['platform'] != user_platform]
    platform_index = platform.index.values
    print(platform_index)
    game_name_check_index = game_name_check.index.values
    game_index = game_a.index.values
    sim_index = cosine_sim[game_index]
    sim_index = sim_index.reshape(-1)
    print(sim_index)
    for i in platform_index:
        sim_index = sim_index[sim_index != i]
    for i in game_name_check_index:
        sim_index = sim_index[sim_index != i]
    return data.loc[sim_index][:top].sort_values(by=['totalscore'], ascending=False)
print(find_sim_game(data, cosine_sim, game_name = "Super Mario Galaxy", game_platform='Wii', user_platform='Switch'))



n_components 값에 따른 Reconstruction Error 측정
Runtime이 매우 기므로 실행할 때 주의할 것

In [ ]:
norm_list = []
num_list = []
for i in range(1, 1001, 100):
    U, s, V = randomized_svd(tfidf_matrix, n_components = i)
    s = np.diag(s)
    X_reconstruction = np.dot(np.dot(U,s), V)
    A = tfidf_matrix.toarray()-X_reconstruction
    k = LA.norm(A)
    num_list.append(i)
    norm_list.append(k)
 

게임 추천 시스템 구현 with FLASK

In [ ]:
!pip install flask
!pip install flask-ngrok
!pip install wget

In [ ]:
import wget

In [ ]:
wget.download('https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz')

ngrok 연결

In [ ]:
!tar -xvf ./ngrok-stable-linux-amd64.tgz

In [ ]:
!ngrok authtoken '2EpvJkNx4hDlzBrjDEpkmtBnihp_2tEVCwiAdxh5nWne33P9H'

Flask를 이용해 웹 애플리케이션 생성

In [ ]:
from flask import Flask, render_template, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)
@app.route('/')
def main():
        return render_template("main.html")
@app.route('/result', methods=['POST', 'GET'])
def recommend():
    if request.method == 'POST':
        game_name = request.form['game_name']
        game_platform = request.form['game_platform']
        user_platform = request.form['user_platform']
        a = find_sim_game(data, cosine_sim, game_name, game_platform, user_platform)
        a = a.drop(['platform', 'totalscore', 'genre', 'developer', 'feature'], axis=1)
        return render_template("result.html",tables = [a.to_html(index=False)])
if __name__== "__main__":
    app.run()